# dateや季節のデータを持ったデータを取得したのでそれに合わせて
# ダミー変換に手を加えたトレインデータ作成
#### ゴール:従来のトレーンデータに月、大会の日数、大会の何日目か、季節の情報をダミー化して追加したカラムの作成

In [1]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression as LR

pd.set_option('display.width',400)#勝手に改行コードを入れられるのを防ぐ

### 前処理

In [30]:
def make_PCA_df(PCA_arr):#PCAで削減したものは二次元配列で帰ってくるので、それをデータフレームにして返す関数
    X=[0]*len(PCA_arr)
    Y=[0]*len(PCA_arr)
    index=0
    for arr in PCA_arr:
        X[index]=arr[0]
        Y[index]=arr[1]
        index+=1
    return pd.DataFrame({'X':X,'Y':Y})

def making_pred_df(df):#配当金、着の情報は切りぬかなくてもうまいことやってくれる。
    pred_race_df=df
    #pred_race_df=pred_race_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除
    result_df=pred_race_df
    result_df=result_df.drop(["racer_1_ID","racer_2_ID","racer_3_ID","racer_4_ID","racer_5_ID","racer_6_ID",],axis=1)#IDはいらないので削除
    result_df=result_df.replace(0.0000,{"racer_1_ave_st_time":0.22})#新人のave_st_timeを0.22に
    result_df=result_df.replace(0.0000,{"racer_2_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_3_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_4_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_5_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_6_ave_st_time":0.22})
    result_df=result_df.replace(0.0000,{"racer_1_doub_win":0.02})#新人の着に絡む確率ave_st_timeを0.02に(新人の半期の偏差から導出)
    result_df=result_df.replace(0.0000,{"racer_2_doub_win":0.02})
    result_df=result_df.replace(0.0000,{"racer_3_doub_win":0.02})
    result_df=result_df.replace(0.0000,{"racer_4_doub_win":0.02})
    result_df=result_df.replace(0.0000,{"racer_5_doub_win":0.02})
    result_df=result_df.replace(0.0000,{"racer_6_doub_win":0.02})
    #配当金と結果の情報を横によけておく。(仮のデータフレームに格納する）=========================================================
    #クラスタリングに使わないカラムを取り除く
    drop_col_names=['result_com','money']
    train_drops_df=pd.DataFrame(columns=drop_col_names)
    train_drops_df['result_com']=result_df['result_com']
    train_drops_df['money']=result_df['money']
    result_df=result_df.drop(drop_col_names,axis=1)
    #==============================================================================================================================
    #ダミー変数化
    #月をダミー化
    empty_arr=[0]*len(result_df)
    for number in np.arange(1,13,1):
        result_df['month_{}'.format(int(number))]=empty_arr
    dummie_df=pd.get_dummies(result_df['month'])#月をダミー化
    for column, val in dummie_df.iteritems():
        result_df['month_{}'.format(int(column))]=val

        
    #季節のダミー化   
    empty_arr=[0]*len(result_df)
    seasons=['sp','su','au','wi']
    for season in seasons:
        result_df['{}'.format(number)]=empty_arr
    dummie_df=pd.get_dummies(result_df['season'])#季節をダミー化
    for column, val in dummie_df.iteritems():
        result_df['{}'.format(column)]=val


    #開催日数のダミー化
    empty_arr=[0]*len(result_df)
    ranges=[1,2,3,4,5,6,7]#年末がおかしくなるっぽいから、一応からむだけ作っておく　
    for number in ranges:
        result_df['range_{}'.format(int(number))]=empty_arr
    dummie_df=pd.get_dummies(result_df['range_date'])#開催日数をダミー化
    for column, val in dummie_df.iteritems():
        result_df['range_{}'.format(int(column))]=val
        
    #開催日数のうちの何日目かのダミー化
    empty_arr=[0]*len(result_df)
    num_dates=[1,2,3,4,5,6,7]
    for number in num_dates:
        result_df['num_date_{}'.format(int(number))]=empty_arr
    dummie_df=pd.get_dummies(result_df['num_date'])#開催日数をダミー化
    for column, val in dummie_df.iteritems():
        result_df['num_date_{}'.format(int(column))]=val
        
    result_df=result_df.drop(['month','season','range_date','num_date'],axis=1)#ダミー化し終わったカラムは削除する
    #レース番号のダミー化===============================================
    result_df_dummie=result_df
    race_dummie_df=pd.get_dummies(result_df_dummie['number_race'])#number_raceをダミー化
    for column, val in race_dummie_df.iteritems():
        result_df_dummie['race_{}'.format(int(column))]=val
    result_df_dummie=result_df_dummie.drop('number_race',axis=1)    
    #===========================新規、性別の取り出し機能が良くなかったため作り直す
    cols=list(result_df_dummie.columns)
    male_cols=[s for s in cols if 'male' in s]#性別を示すカラムを取り出す
    empty_arr=[0]*len(result_df_dummie)
    for col in male_cols:
        for number in np.arange(0,2,1):
              result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr
        male_dummie_df=pd.get_dummies(result_df_dummie[col])#性別をダミー化
        for column, val in male_dummie_df.iteritems():
              result_df_dummie['{}_{}'.format(col,int(column))]=val
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)

    cols=list(result_df_dummie.columns)
    moter_cols=[s for s in cols if '_mo' in s]#モーター番号を示すカラムを取り出す
    boat_cols=[s for s in cols if '_bo' in s]#ボート番号を示すカラムを取り出す
    #boat もmoterも番号は1~99とする
    numbers=np.arange(1, 100, 1)
    empty_arr=[0]*len(result_df_dummie)
    for col in moter_cols:
        for number in numbers:
              result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr
        moter_dummie_df=pd.get_dummies(result_df_dummie[col])#モータ番号をダミー化
        for column, val in moter_dummie_df.iteritems():
              result_df_dummie['{}_{}'.format(col,int(column))]=val
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)

    #boat番号をダミー化
    for col in boat_cols:
        for number in numbers:
              result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr
        boat_dummie_df=pd.get_dummies(result_df_dummie[col])#boat番号をダミー化
        for column, val in boat_dummie_df.iteritems():
              result_df_dummie['{}_{}'.format(col,int(column))]=val
        result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)
    result_df=result_df_dummie



    #クラスタリング
    #分けてみるクラスタの数は[8,10]の2個
    #cluster_target_df　　trainのデータからリザルトと配当金を取り除いたもの
    target_num_cluster=[8,10]
    #test_clustaring_df=train_has_PCA_df
    clustar_target_df=result_df
    clustaring_df=clustar_target_df
    for num_cluster in target_num_cluster:
        pred = KMeans(random_state=1,n_clusters=num_cluster).fit_predict(clustar_target_df)
        clustaring_df['num={}'.format(num_cluster)]=pred

    clustaring_df['result_com']=train_drops_df['result_com']#正解ラベルを戻してあげる
    clustaring_df['money']=train_drops_df['money']#配当金を戻してあげる
    model_df=clustaring_df

    return model_df

In [48]:
place='tokuyama'

result_df=pd.read_csv('../csv/train_{}_date.csv'.format(place),encoding='utf-8')
result_df=result_df.drop(["Unnamed: 0"],axis=1)

result_df=result_df.drop(['date','day'],axis=1)#現状使わないカラムは削除する

date_cols=['year','month','num_date','range_date','season']
date_df=pd.DataFrame(index=result_df.index)
for col in date_cols:
    date_df[col]=result_df[col]
result_df=result_df.drop('year',axis=1)#yearカラムはデータの切り抜きで使うが学習では使わないがので削除（関数内でdropされないから）
#result_df=result_df.drop(col,axis=1)
#money_col=result_df['money']
#result_df=result_df.drop('money',axis=1)#配当金情報の削除
#result_col=result_df['result_com']
#result_df=result_df.drop('result_com',axis=1)#着の組み合わせ
result_df=making_pred_df(result_df)#モデルに突っ込むだけの状態に加工（新人選手の情報、欠損地加工とダミー化）
result_df.to_csv('train_df.csv')
df=result_df
### 学習データのベースを作成(validとtrainに分割)
#学習、テストデータ切り分け(2019,2020のデータを検証用データに、ほかは学習)
df['year']=date_df['year']
valid_df = df[(df['year']==2019) | ((df['year']==2020) )]#2019,2020のデータを検証用データに。
train_df =  df[(df['year']!=2019) & ((df['year']!=2020) )]#そのほかを学習データに
#学習データを切り分けたらyearはいらないから削除する
valid_df=valid_df.drop(['year'],axis=1)
train_df=train_df.drop(['year'],axis=1)

train_money=pd.Series(train_df['money'])
valid_money=pd.Series(valid_df['money'])
valid_total_df=valid_df.copy()#あとで分析用に全部のまとめ先のDf
train_total_df=train_df.copy()#あとで分析用に全部のまとめ先のDf


0         1160.0
1         2960.0
2        21490.0
3         5100.0
4         1980.0
          ...   
15166    15230.0
15167     1350.0
15168      640.0
15169     5160.0
15170     1310.0
Name: money, Length: 15171, dtype: float64

In [46]:
valid_df


,racer_1_age,racer_1_ave_st,racer_1_doub,racer_1_rank,racer_2_age,racer_2_ave_st,racer_2_doub,racer_2_rank,racer_3_age,racer_3_ave_st,...,racer_6_bo_95,racer_6_bo_96,racer_6_bo_97,racer_6_bo_98,racer_6_bo_99,num=8,num=10,result_com,money,year
15171,31.0,0.16,0.453,4,27.0,0.16,0.170,1,51.0,0.20,...,0,0,0,0,0,7,3,7,5780.0,2019
15172,52.0,0.18,0.328,3,34.0,0.21,0.121,1,23.0,0.12,...,0,0,0,0,0,0,8,5,790.0,2019
15173,38.0,0.16,0.525,4,57.0,0.22,0.078,2,34.0,0.16,...,0,0,0,0,0,6,7,6,390.0,2019
15174,31.0,0.19,0.344,3,38.0,0.18,0.450,4,34.0,0.20,...,0,0,0,0,0,3,9,31,3060.0,2019
15175,32.0,0.20,0.245,2,47.0,0.15,0.388,3,32.0,0.17,...,0,0,0,0,0,3,9,1,1360.0,2019
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19337,51.0,0.19,0.156,2,42.0,0.13,0.508,4,44.0,0.17,...,0,0,0,0,0,2,0,83,54960.0,2020
19338,44.0,0.17,0.426,3,33.0,0.15,0.308,2,49.0,0.16,...,0,0,0,0,0,2,0,14,650.0,2020
19339,47.0,0.18,0.464,4,25.0,0.14,0.271,2,31.0,0.16,...,0,0,0,0,0,0,8,10,3930.0,2020
19340,46.0,0.17,0.247,2,33.0,0.15,0.510,4,42.0,0.13,...,0,0,0,0,0,4,4,25,1430.0,2020


In [47]:
train_df

,racer_1_age,racer_1_ave_st,racer_1_doub,racer_1_rank,racer_2_age,racer_2_ave_st,racer_2_doub,racer_2_rank,racer_3_age,racer_3_ave_st,...,racer_6_bo_95,racer_6_bo_96,racer_6_bo_97,racer_6_bo_98,racer_6_bo_99,num=8,num=10,result_com,money,year
0,35.0,0.14,0.541,4,40.0,0.19,0.287,2,30.0,0.19,...,0,0,0,0,0,3,9,6,1160.0,2012
1,46.0,0.18,0.450,4,27.0,0.18,0.301,2,31.0,0.15,...,0,0,0,0,0,0,8,3,2960.0,2012
2,43.0,0.16,0.480,3,35.0,0.21,0.255,2,42.0,0.17,...,0,0,0,0,0,1,1,44,21490.0,2012
3,27.0,0.17,0.506,4,38.0,0.20,0.188,2,41.0,0.17,...,0,0,0,0,0,3,9,62,5100.0,2012
4,32.0,0.21,0.439,3,25.0,0.17,0.536,4,45.0,0.16,...,0,0,0,0,0,0,8,25,1980.0,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15166,37.0,0.15,0.142,2,27.0,0.17,0.069,2,45.0,0.17,...,0,0,0,0,0,1,1,16,15230.0,2018
15167,54.0,0.16,0.320,2,23.0,0.16,0.103,1,53.0,0.19,...,0,0,0,0,0,1,1,9,1350.0,2018
15168,49.0,0.15,0.487,4,51.0,0.19,0.428,1,51.0,0.18,...,0,0,0,0,0,6,2,2,640.0,2018
15169,47.0,0.14,0.425,4,49.0,0.17,0.287,2,33.0,0.16,...,0,0,0,0,0,2,2,21,5160.0,2018


#### モデルの性能評価用のデータフレーム (モデルを使うかを判断する閾値を収納する)

In [41]:
#出現数の分布
result_com_s=valid_df['result_com'].value_counts()
result_com_s=result_com_s.sort_index()
gain_mean=valid_df.groupby('result_com')['money'].mean()
gain_mean=gain_mean.sort_index()

gain_median=valid_df.groupby('result_com')['money'].median()
gain_median=gain_median.sort_index()
result_com_df=pd.DataFrame({'result_com':result_com_s.index,
                            'result_com_num':result_com_s.values,
                            'result_com_per':result_com_s.values/sum(result_com_s.values)*100,
                            'gain_mean':gain_mean.values,
                            'gain_median':gain_median.values,})
result_com_df=result_com_df.iloc[0:28]#探索的に探すにも最後のほうは役にモデルなのはわかっているため
#result_com_df=result_com_df.set_index('result_com')

In [42]:
result_com_df

,result_com,result_com_num,result_com_per,gain_mean,gain_median
0,1,327,7.839847,1101.773700,770.0
1,2,270,6.473268,1296.814815,870.0
2,3,184,4.411412,1945.163043,1260.0
3,4,107,2.565332,2463.644860,1720.0
4,5,238,5.706066,1335.546218,1020.0
5,6,223,5.346440,1274.304933,940.0
6,7,182,4.363462,1605.494505,975.0
7,8,104,2.493407,2787.788462,2270.0
8,9,176,4.219612,1629.034091,1050.0
9,10,170,4.075761,1715.176471,1220.0


## モデリング(探索的にパラメータを決める)

In [27]:
#result_com=1
#depth=5
model_score_df=pd.DataFrame(columns=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])
for result_com_number in result_com_df['result_com'].values:
    print(result_com_number)
    result_com=result_com_number
    #result_comごとの閾値の決定========================================================================
    
    gain_th=10#利益率の閾値
    result_s=result_com_df[result_com_df['result_com']==result_com]
    buy_accuracy_th=result_s['result_com_per'].values[0]*1.1#買ったうちの的中率の閾値
    num_tp_th=result_s['result_com_num'].values[0]*0.2#あたった回数の閾値(出現回数の20%が的中)
    #===============================================================================
    #学習データのラベル変換==========================================================
    result_train_df=train_df.copy() 
    result_arr=[0]*len(result_train_df)
    i=0
    for result in result_train_df['result_com']:
        if ((result==result_com)):
            result_arr[i]=1
        else:
            result_arr[i]=0
        i+=1
    result_train_df['result_com']=result_arr
    result_valid_df=valid_df.copy() 
    result_arr=[0]*len(result_valid_df)
    i=0
    for result in result_valid_df['result_com']:
        if ((result==result_com)):
            result_arr[i]=1
        else:
            result_arr[i]=0
        i+=1

    result_valid_df['result_com']=result_arr

    result_train_df['money']=train_money
    result_valid_df['money']=valid_money
    #学習データラベル変換終わり============================================

    for_arr=np.arange(1,100,1)
    accuracy_arr=[0]*len(for_arr)
    target_per_arr=[0]*len(for_arr)
    pred_0=[0]*len(for_arr)
    gain_arr=[0]*len(for_arr)
    model_gain_arr=[0]*len(result_valid_df)
    valid_gain_arr=valid_money.values

    depths_arr=[4,5,6,7,8]
    for depth in depths_arr:
        for sum_target_per in tqdm(for_arr):
            
            index=sum_target_per-1
            target_per=50+sum_target_per
            target_per_arr[index]=target_per

            #モデルの評価指標値を格納するseries======================
            model_score_s=pd.Series(index=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])
            model_score_s['target_com']=result_com#目標としているresult_comラベル番号
            model_score_s['depth']=depth#ハイパーパラメータ＿木の深さ
            model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
            #======================
            #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
            # 一層目の判別機のtrainデータ　:terget_result_df
            target_df=result_train_df#ベースのデータフレームをコピー
            target_df=target_df.sample(frac=1, random_state=1)#シャッフル、時系列の偏りを無くす
            target_1_df=target_df[target_df['result_com']==1]
            len_1=len(target_1_df)
            target_0_df=target_df[target_df['result_com']==0]
            len_0=len(target_0_df)
            target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0]#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
            target_train_df=pd.concat([target_1_df, target_0_df])
            #以下学習パート======================================================
            target_x_train=target_train_df.drop('money',axis=1)
            target_x_train=target_x_train.drop('result_com',axis=1)
            target_y_train=target_train_df['result_com']
            #テストデータ
            target_y_valid=result_valid_df['result_com']
            target_x_valid=result_valid_df.drop('money',axis=1)
            target_x_valid=target_x_valid.drop('result_com',axis=1)
            RF = RandomForestClassifier(random_state=1,n_estimators=1000,max_depth=depth)
            RF = RF.fit(target_x_train,target_y_train)


            #以下精度検証(１の正答率のみ調査)

            # 未知データに対する予測値
            predict_y_valid = RF.predict(target_x_valid)

            #[1]の正答率を見る
            pred_valid_df=pd.DataFrame({'pred':predict_y_valid
                                      , 'valid':target_y_valid})
            num_1=len(pred_valid_df[pred_valid_df['valid']==1])
            count=0
            #追加　配当金の情報も考慮する。
            gain_index=0
            model_gain_arr=[0]*len(result_valid_df)
            for _, s in pred_valid_df.iterrows():
                if ((s['pred']==1) and (s['valid']==1)):
                    count+=1#的中回数
                    model_gain_arr[gain_index]=valid_gain_arr[gain_index]
                gain_index+=1
            #print('test accyracy: {}'.format((count/num_1)*100))
            gain_arr[index]=sum(model_gain_arr)
            accuracy_arr[index]=(count/num_1)*100
            try:
                pred_0[index]=pred_valid_df['pred'].value_counts()[0]
            except:
                pred_0[index]=0
            #scoreのseriesに情報書き込み==================
            model_score_s['総収益']=sum(model_gain_arr)
            model_score_s['投資金額']=100*sum(predict_y_valid)
            model_score_s['出現数']=sum(target_y_valid)
            model_score_s['購買予測数']=sum(predict_y_valid)
            model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
            model_score_s['購買的中率']=(count/sum(predict_y_valid))*100
            model_score_s['的中数']=count
            model_score_df=model_score_df.append(model_score_s,ignore_index=True)
            
#モデルの「スコアを保存        
model_score_df.to_csv('model_score.csv', encoding='utf_8_sig')
            
            

  0%|                                                                                                                                                                                                                | 0/99 [00:00<?, ?it/s]

1.0


 91%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉                  | 90/99 [05:14<00:34,  3.81s/it]c:\users\nabe0\desktop\nabepy\env\lib\site-packages\ipykernel_launcher.py:114: RuntimeWarning: invalid value encountered in double_scalars
c:\users\nabe0\desktop\nabepy\env\lib\site-packages\ipykernel_launcher.py:115: RuntimeWarning: invalid value encountered in longlong_scalars
 93%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉              | 92/99 [05:22<00:26,  3.83s/it]c:\users\nabe0\desktop\nabepy\env\lib\site-packages\ipykernel_launcher.py:114: RuntimeWarning: invalid value encountered in double_scalars
c:\users\nabe0\desktop\nabepy\env\lib\site-packages\ipykernel_launcher.py:115: RuntimeWarning: invalid value 

2.0


 93%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉              | 92/99 [06:05<00:30,  4.32s/it]c:\users\nabe0\desktop\nabepy\env\lib\site-packages\ipykernel_launcher.py:114: RuntimeWarning: invalid value encountered in double_scalars
c:\users\nabe0\desktop\nabepy\env\lib\site-packages\ipykernel_launcher.py:115: RuntimeWarning: invalid value encountered in longlong_scalars
 95%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▉          | 94/99 [06:14<00:21,  4.34s/it]c:\users\nabe0\desktop\nabepy\env\lib\site-packages\ipykernel_launcher.py:114: RuntimeWarning: invalid value encountered in double_scalars
c:\users\nabe0\desktop\nabepy\env\lib\site-packages\ipykernel_launcher.py:115: RuntimeWarning: invalid value 

...

  0%|                                                                                                                                                                                                                | 0/99 [00:00<?, ?it/s]

118.0


  0%|                                                                                                                                                                                                                | 0/99 [00:00<?, ?it/s]

119.0


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 99/99 [04:10<00:00,  2.53s/it]


## モデルのスコアを保存

In [28]:
model_score_df.to_csv('model_score.csv', encoding='utf_8_sig')

### モデリング用＿動作確認

In [29]:
"""
result_com=1
depth=5
model_score_df=pd.DataFrame(columns=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])

#result_comごとの閾値の決定========================================================================
gain_th=10#利益率の閾値
result_s=result_com_df[result_com_df['result_com']==result_com]
buy_accuracy_th=result_s['result_com_per'].values[0]*1.1#買ったうちの的中率の閾値
num_tp_th=result_s['result_com_num'].values[0]*0.2#あたった回数の閾値(出現回数の20%が的中)
#===============================================================================
#学習データのラベル変換==========================================================
result_train_df=train_df
result_arr=[0]*len(result_train_df)
i=0
for result in result_train_df['result_com']:
    if ((result==result_com)):
        result_arr[i]=1
    else:
        result_arr[i]=0
    i+=1
result_train_df['result_com']=result_arr

result_valid_df=valid_df
result_arr=[0]*len(result_valid_df)
i=0
for result in result_valid_df['result_com']:
    if ((result==result_com)):
        result_arr[i]=1
    else:
        result_arr[i]=0
    i+=1

result_valid_df['result_com']=result_arr

result_train_df['money']=train_money
result_valid_df['money']=valid_money
#学習データラベル変換終わり============================================

for_arr=np.arange(1,100,1)
accuracy_arr=[0]*len(for_arr)
target_per_arr=[0]*len(for_arr)
pred_0=[0]*len(for_arr)
gain_arr=[0]*len(for_arr)
model_gain_arr=[0]*len(result_valid_df)
valid_gain_arr=valid_money.values
for sum_target_per in for_arr:
    
    index=sum_target_per-1
    target_per=50+sum_target_per
    target_per_arr[index]=target_per
    
    #モデルの評価指標値を格納するseries======================
    model_score_s=pd.Series(index=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])
    model_score_s['target_com']=result_com#目標としているresult_comラベル番号
    model_score_s['depth']=depth#ハイパーパラメータ＿木の深さ
    model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
    #======================
    #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
    # 一層目の判別機のtrainデータ　:terget_result_df
    target_df=result_train_df#ベースのデータフレームをコピー
    target_df=target_df.sample(frac=1, random_state=1)#シャッフル、時系列の偏りを無くす
    target_1_df=target_df[target_df['result_com']==1]
    len_1=len(target_1_df)
    target_0_df=target_df[target_df['result_com']==0]
    len_0=len(target_0_df)
    target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0]#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
    target_train_df=pd.concat([target_1_df, target_0_df])
    #以下学習パート======================================================
    target_x_train=target_train_df.drop('money',axis=1)
    target_x_train=target_x_train.drop('result_com',axis=1)
    target_y_train=target_train_df['result_com']
    #テストデータ
    target_y_valid=result_valid_df['result_com']
    target_x_valid=result_valid_df.drop('money',axis=1)
    target_x_valid=target_x_valid.drop('result_com',axis=1)
    RF = RandomForestClassifier(random_state=1,n_estimators=1000,max_depth=depth)
    RF = RF.fit(target_x_train,target_y_train)


    #以下精度検証(１の正答率のみ調査)

    # 未知データに対する予測値
    predict_y_valid = RF.predict(target_x_valid)

    #[1]の正答率を見る
    pred_valid_df=pd.DataFrame({'pred':predict_y_valid
                              , 'valid':target_y_valid})
    num_1=len(pred_valid_df[pred_valid_df['valid']==1])
    count=0
    #追加　配当金の情報も考慮する。
    gain_index=0
    model_gain_arr=[0]*len(result_valid_df)
    for _, s in pred_valid_df.iterrows():
        if ((s['pred']==1) and (s['valid']==1)):
            count+=1#的中回数
            model_gain_arr[gain_index]=valid_gain_arr[gain_index]
        gain_index+=1
    #print('test accyracy: {}'.format((count/num_1)*100))
    gain_arr[index]=sum(model_gain_arr)
    accuracy_arr[index]=(count/num_1)*100
    try:
        pred_0[index]=pred_valid_df['pred'].value_counts()[0]
    except:
        pred_0[index]=0
    #scoreのseriesに情報書き込み==================
    model_score_s['総収益']=sum(model_gain_arr)
    model_score_s['投資金額']=100*sum(predict_y_valid)
    model_score_s['出現数']=sum(target_y_valid)
    model_score_s['購買予測数']=sum(predict_y_valid)
    model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
    model_score_s['購買的中率']=(count/sum(predict_y_valid))*100
    model_score_s['的中数']=count
    model_score_df=model_score_df.append(model_score_s,ignore_index=True)

"""

"\nresult_com=1\ndepth=5\nmodel_score_df=pd.DataFrame(columns=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])\n\n#result_comごとの閾値の決定========================================================================\ngain_th=10#利益率の閾値\nresult_s=result_com_df[result_com_df['result_com']==result_com]\nbuy_accuracy_th=result_s['result_com_per'].values[0]*1.1#買ったうちの的中率の閾値\nnum_tp_th=result_s['result_com_num'].values[0]*0.2#あたった回数の閾値(出現回数の20%が的中)\n#===============================================================================\n#学習データのラベル変換==========================================================\nresult_train_df=train_df\nresult_arr=[0]*len(result_train_df)\ni=0\nfor result in result_train_df['result_com']:\n    if ((result==result_com)):\n        result_arr[i]=1\n    else:\n        result_arr[i]=0\n    i+=1\nresult_train_df['result_com']=result_arr\n\nresult_valid_df=valid_df\nresult_arr=[0]*len(result_valid_df)\ni=0\nfor result in result_valid_df['result_com']:

In [12]:
100*sum(predict_y_valid)

5900

In [13]:
(sum(model_gain_arr)/100*sum(predict_y_valid))

3262.7

In [54]:
model_score_df[:30]

,target_com,depth,target_per,総収益,投資金額,出現数,購買予測数,利益率,購買的中率,的中数
0,1.0,4.0,51.0,256020.0,314900.0,234.0,3149.0,81.302001,7.430930,234.0
1,1.0,4.0,52.0,256020.0,314900.0,234.0,3149.0,81.302001,7.430930,234.0
2,1.0,4.0,53.0,256020.0,314900.0,234.0,3149.0,81.302001,7.430930,234.0
3,1.0,4.0,54.0,256020.0,314900.0,234.0,3149.0,81.302001,7.430930,234.0
4,1.0,4.0,55.0,256020.0,314900.0,234.0,3149.0,81.302001,7.430930,234.0
5,1.0,4.0,56.0,256020.0,314900.0,234.0,3149.0,81.302001,7.430930,234.0
6,1.0,4.0,57.0,256020.0,314900.0,234.0,3149.0,81.302001,7.430930,234.0
7,1.0,4.0,58.0,256020.0,314900.0,234.0,3149.0,81.302001,7.430930,234.0
8,1.0,4.0,59.0,256020.0,314900.0,234.0,3149.0,81.302001,7.430930,234.0
9,1.0,4.0,60.0,256020.0,314900.0,234.0,3149.0,81.302001,7.430930,234.0


In [20]:
result_s

,result_com,result_com_num,result_com_per,gain_mean,gain_median
0,1.0,234,7.43093,1094.102564,785.0
